In [1]:
import sys
import matplotlib.pyplot as plt
import scprep
import pandas as pd
import numpy as np
import scanpy as sc

In [2]:
data = np.load('../data/eb_subset_all.npz')

In [20]:
raw_data = sc.read_h5ad('../data/old/eb_hv.h5ad')

In [14]:
data_old = np.load('../data/old/eb_hv_pca_subset.npy')

In [12]:
raw_data.obsm['X_phate']

array([[ 0.01935549, -0.00797983],
       [ 0.04288998,  0.02949095],
       [ 0.04142842,  0.0280109 ],
       ...,
       [-0.02111476, -0.00016555],
       [-0.02158279, -0.00016536],
       [-0.01735641, -0.00157353]])

In [18]:
(data['data'] == data_old).all()

True

In [19]:
data_old.shape

(3000, 50)

In [21]:
np.random.seed(42)
subset_id = np.random.choice(raw_data.shape[0], size=3000, replace=False)
X_pca_subset = raw_data.obsm['X_pca'][subset_id,:]

In [23]:
(data['data'] == X_pca_subset).all()

True

In [29]:
raw_data.varm['PCs'].shape

(1684, 50)

In [32]:
pca_loadings = raw_data.varm['PCs']  # PCA loadings
data_mean = np.mean(raw_data.X, axis=0)  # Compute the mean of the original data

In [33]:
pca_data = data['data']
original_space_data = np.dot(pca_data, pca_loadings.T) + data_mean
original_space_data = np.asarray(original_space_data)


In [35]:
raw_data.X

<16821x1684 sparse matrix of type '<class 'numpy.float64'>'
	with 3446327 stored elements in Compressed Sparse Row format>

In [36]:
import sys
sys.path.append('../src')
from evaluation import pearson_correlation_corresponding

In [44]:
actual_data = raw_data[subset_id,:].X.toarray()

In [57]:
corrs = pearson_correlation_corresponding(original_space_data, actual_data)

In [59]:
corrs.mean()

0.2918863630596089

In [69]:
x_magic

array([[-1.79874788e-01,  1.77552745e-01,  1.24380966e+00, ...,
        -1.49910190e-01,  2.76596831e-01, -4.63554424e-01],
       [ 6.60449411e-01, -1.01056013e+00, -1.06202805e+00, ...,
         1.33767903e-01,  3.25151256e-01, -5.40944890e-02],
       [-3.16819965e-01, -4.75869277e-01, -1.29266071e+00, ...,
         1.36273339e-01,  5.60034402e-03,  2.56478554e-01],
       ...,
       [-4.60610591e+00, -1.56684569e+00,  5.83325569e+00, ...,
        -3.73826670e-01, -9.53925076e-02, -2.55217638e-02],
       [-4.07017703e+00, -6.55004576e-01,  1.65875036e-01, ...,
        -1.62177526e-01, -1.72656096e-01, -3.25158076e-02],
       [-1.81190124e+00, -1.53630470e+00, -1.11290874e+00, ...,
         5.26763537e-02, -3.75978377e-01,  1.35822816e-01]])

In [70]:
pca_data

array([[-1.7016331e+00,  8.8023923e-02,  2.0021570e+00, ...,
        -1.3836890e+00,  3.6277589e-01,  1.3107024e-01],
       [-2.1301298e-01, -2.3534687e+00,  5.5244428e-01, ...,
         5.4930478e-01,  1.3934861e+00,  1.7196587e-01],
       [-1.3155088e+00, -1.1787565e+00, -9.6000332e-01, ...,
         1.3089414e+00, -7.7756536e-01,  1.2356104e+00],
       ...,
       [-4.7607551e+00, -1.4115350e+00,  4.5781436e+00, ...,
        -1.0765858e+00, -9.8167352e-02,  8.4981084e-01],
       [-4.9118619e+00, -6.2910384e-01, -2.7711913e-01, ...,
        -6.2702727e-01, -1.4723185e+00, -5.5469060e-01],
       [-5.9857810e-01, -2.4779913e+00, -1.5524262e+00, ...,
         6.0734707e-01,  5.1273879e-02, -1.5904061e-03]], dtype=float32)

In [88]:
import magic
magic_op = magic.MAGIC()
x_magic = magic_op.fit_transform(pca_data)
original_space_magic = np.dot(x_magic, pca_loadings.T) + data_mean
original_space_magic = np.asarray(original_space_magic)

Calculating MAGIC...
  Running MAGIC on 3000 cells and 50 genes.
  Calculating graph and diffusion operator...
    Calculating KNN search...
    Calculated KNN search in 0.67 seconds.
    Calculating affinities...
    Calculated affinities in 0.65 seconds.
  Calculated graph and diffusion operator in 1.32 seconds.
  Calculating imputation...
Calculated MAGIC in 1.33 seconds.


In [89]:
corrs = pearson_correlation_corresponding(original_space_magic, original_space_data)

In [90]:
corrs.mean()

0.8114686766133132

In [71]:
datapcs = np.load('../data/eb_pcs.npz')

In [76]:
np.savez('../data/eb_all_pcs_correct.npz', pca_loadings=pca_loadings, data_mean=data_mean)